## Franks Numbers

In [10]:
# libraries 
import numpy as np       
import pandas as pd
import matplotlib.pyplot as plt                        
from iminuit import Minuit                          
import sys  

# external libraries
sys.path.append('../') 
import AdvAppStatFunctions as aas

# setting for plotting
import seaborn as sns
plt.style.use('seaborn-white')
plt.rcParams['font.size'] = 14
plt.rcParams['xtick.labelsize']=12
plt.rcParams['ytick.labelsize']=12

### Parsing data

In [146]:
# with help from Tania Kozynets' notebook

# read data
data = pd.read_table('FranksNumbers.txt', header=1, sep='\s+', names = ['x','y','nan'])
# sep='\s+' defines the separator as being one single white space or more

# remove last column since this is only nans
data = data.drop(['nan'], axis=1)   #axis = 1 removes data in column

# check if element in x-column is numeric. returns an array with true/false statements
check_if_numeric = np.array([data['x'][i].isnumeric() for i in range(len(data))])

# if element is not numeric, it must be the start of the header. now, select the rows with headers 
header_rows = np.where(check_if_numeric == False)[0]
N_dataset = len(header_rows)+1

# create array with all breaking rows (-1 to include the first header and len(data) to include the last row)
breaking_rows = np.append(-1,header_rows)
breaking_rows = np.append(breaking_rows,len(data))

# storing the individual datasets in one big dictionary 
data_sorted = []

for dataset_number in range(N_dataset):
    data_sorted.append(np.array(data[breaking_rows[dataset_number]+1: \
                                  breaking_rows[dataset_number+1]].astype('float')))

# backslash creates implicit line joining

### Calculate the mean and variance for each data set in the file

In [136]:
def variance(data, bias = False):
    N = len(data)
    if bias: return sum((data-np.average(data))**2) / N   # biased
    else: return sum((data-np.average(data))**2) / (N-1)  # unbiased

In [331]:
for i in range(N_dataset):
    n_dataset = i+1
    mean = np.average(data_sorted[i][1], axis=0)
    biased_variance = variance(data_sorted[0][:,1], bias = True)
    unbiased_variance = variance(data_sorted[0][:,1])
    
    print(f'--- Dataset {n_dataset} --- \
    \nMean = {mean:.2f} \
    \nBiased variance = {biased_variance:.2f} \
    \nUnbiased variance = {unbiased_variance:.2f} \n')

--- Dataset 1 ---     
Mean = 7.47     
Biased variance = 3.75     
Unbiased variance = 4.13 

--- Dataset 2 ---     
Mean = 8.07     
Biased variance = 3.75     
Unbiased variance = 4.13 

--- Dataset 3 ---     
Mean = 7.38     
Biased variance = 3.75     
Unbiased variance = 4.13 

--- Dataset 4 ---     
Mean = 6.88     
Biased variance = 3.75     
Unbiased variance = 4.13 

--- Dataset 5 ---     
Mean = 6.88     
Biased variance = 3.75     
Unbiased variance = 4.13 



### Using the eq. $y=x \cdot 0.48 + 3.02$, calculate the Pearson’s $\chi^2$ for each data set

In [272]:
def theoretical(x):
    return x*0.48+3.02

# extended dataset with the theoretical value in the third column
data_extended = []

for i in range(N_dataset): #0-4
    dataset_extended = []
    
    for j in range(len(data_sorted[i][:,0])):
        dataset_extended.append(np.append(data_sorted[i][j], theoretical(data_sorted[i][j][0])).tolist())
        
    data_extended.append(np.array(dataset_extended))

In [313]:
def chi_square(y_exp, y_obs, y_err):
    return np.sum((y_exp-y_obs)**2/y_err**2)

In [322]:
chi2_sqrt_err = []
chi2_fixed_err = []

for i in range(N_dataset):
    y_exp = data_extended[i][:,2]
    y_obs = data_extended[i][:,1]
     
    # uncertainty is the squareroot of the expected value
    y_err = np.sqrt(y_exp)  
    chi2_sqrt_err.append(chi_square(y_exp, y_obs, y_err))
    
    # uncertainty is the squareroot of the expected value
    y_err = 1.22
    chi2_fixed_err.append(chi_square(y_exp, y_obs, y_err))

In [335]:
for i in range(N_dataset):
    n_dataset = i+1
    
    print(f'--- Dataset {n_dataset} --- \
    \nChi2 (w/ pm sqrt uncertainty) \= {chi2_sqrt_err[i]:.3f} \
    \nChi2 (w/ pm 1.22 uncertainty) = {chi2_fixed_err[i]:.3f} \n')

--- Dataset 1 ---     
Chi2 (w/ pm sqrt uncertainty) \= 1.887     
Chi2 (w/ pm 1.22 uncertainty) = 9.468 

--- Dataset 2 ---     
Chi2 (w/ pm sqrt uncertainty) \= 2.072     
Chi2 (w/ pm 1.22 uncertainty) = 9.477 

--- Dataset 3 ---     
Chi2 (w/ pm sqrt uncertainty) \= 1.555     
Chi2 (w/ pm 1.22 uncertainty) = 9.460 

--- Dataset 4 ---     
Chi2 (w/ pm sqrt uncertainty) \= 2.043     
Chi2 (w/ pm 1.22 uncertainty) = 9.454 

--- Dataset 5 ---     
Chi2 (w/ pm sqrt uncertainty) \= 7.556     
Chi2 (w/ pm 1.22 uncertainty) = 37.858 



The sqrt(y_exp) - uncertainty shows better agreement with the data since this uncertainty is larger than 1.22. However, since N is small in all five data sets this is a bad way to estimate the uncertainty. This is only ok when N is large. 